In [86]:
import pandas as pd
import numpy as np
from fredapi import Fred
import yfinance as yf
import glob

In [87]:
start_date = '2023-09-01'
end_date = '2024-09-01'

In [88]:
# target stock data: close stock price
# competitor stock price: competitor stock close price
# macroeconomic indicators: ADS
# industry related data: FRED('CBBTCUSD', 'CBETHUSD')
# financial & investment data: FF5, FRED('SP500', 'NASDAQ', 'DOWJONES')


In [89]:
# NVDA Stock Price
NVDA_stock_price = yf.download("NVDA", start_date, end_date)['Adj Close']
NVDA_stock_price_df = pd.DataFrame(NVDA_stock_price)
NVDA_stock_price_df.index.name = 'Date'
NVDA_stock_price_df.index = NVDA_stock_price_df.index.strftime('%y%m%d')
# Display descriptive statistics for stock prices
print(NVDA_stock_price_df.describe())
NVDA_stock_price_df.to_csv('NVDA_stock_price.csv')

[*********************100%***********************]  1 of 1 completed

Ticker        NVDA
count   251.000000
mean     79.092747
std      30.492500
min      40.313805
25%      48.141552
50%      79.504669
75%     104.851032
max     135.568405


In [90]:
# Competitors stock price
competitors = ['AMD', 'INTC', 'QCOM', 'AAPL', 'AVGO']
competitors_stock_price = yf.download(competitors, start_date, end_date)['Adj Close']
competitors_stock_price_df = pd.DataFrame(competitors_stock_price)
competitors_stock_price_df.index.name = 'Date'
competitors_stock_price_df.index = competitors_stock_price_df.index.strftime('%y%m%d')
print(competitors_stock_price_df.describe())
competitors_stock_price_df.to_csv('competitors_stock_price.csv')

[*********************100%***********************]  5 of 5 completed

Ticker        AAPL         AMD        AVGO        INTC        QCOM
count   251.000000  251.000000  251.000000  251.000000  251.000000
mean    189.827509  147.360677  122.638521   36.445328  154.915609
std      18.559048   28.342445   27.476383    7.405938   32.302531
min     164.405121   93.669998   79.609314   18.990000  102.638420
25%     174.560921  122.160000   95.772259   30.898989  127.521931
50%     185.949036  152.270004  125.089081   35.935158  156.338058
75%     195.181770  167.779999  139.818703   42.780405  174.028770
max     234.290756  211.380005  181.710449   50.089161  225.922470


In [91]:
# ADS Index Data
ADS = pd.read_excel('ADS_Index_Most_Current_Vintage.xlsx')
ADS['Date'] = pd.to_datetime(ADS['Date'], format='%Y:%m:%d')
ADS = ADS[(ADS['Date'] >= pd.to_datetime(start_date)) & (ADS['Date'] <= pd.to_datetime(end_date))]
ADS['Date'] = ADS['Date'].dt.strftime('%y%m%d')
ADS = ADS[ADS['Date'].isin(NVDA_stock_price_df.index)]
ADS.set_index('Date', inplace=True)

# Display descriptive statistics for ADS Index
print(ADS.head(10))


        ADS_Index
Date             
230901   0.060395
230905   0.070788
230906   0.071438
230907   0.071330
230908   0.070466
230911   0.062606
230912   0.057263
230913   0.050435
230914   0.042119
230915   0.032312


In [92]:
# Fama French 5 Factors
FF5 = pd.read_csv('FF5.csv')
FF5.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
FF5['Date'] = pd.to_datetime(FF5['Date'], format='%Y%m%d')
FF5 = FF5[(FF5['Date'] >= pd.to_datetime(start_date)) & (FF5['Date'] <= pd.to_datetime(end_date))]
FF5['Date'] = FF5['Date'].dt.strftime('%y%m%d')
FF5 = FF5[FF5['Date'].isin(NVDA_stock_price_df.index)]
FF5.set_index('Date', inplace=True)

# Display descriptive statistics for Fama French 5 Factors
print(FF5.head(10))
FF5.to_csv('filtered_FF5.csv')

        Mkt-RF   SMB   HML   RMW   CMA     RF
Date                                         
230901    0.26  0.84  0.58 -0.03 -0.20  0.021
230905   -0.54 -1.58 -0.86 -0.30 -0.47  0.021
230906   -0.71  0.01 -0.18 -0.45  0.10  0.021
230907   -0.40 -0.69  0.03 -0.20  0.02  0.021
230908    0.12 -0.51  0.40  0.28  0.01  0.021
230911    0.66 -0.47 -0.54 -0.49 -0.33  0.021
230912   -0.59  0.21  0.99  0.25  0.23  0.021
230913    0.03 -0.87 -0.22  0.25 -0.36  0.021
230914    0.84  0.49  0.86  0.82  0.02  0.021
230915   -1.21  0.21  0.25 -0.33  0.50  0.021


In [93]:
# Initialize Fred API
fred = Fred(api_key='96b8dcff4c5efa52d89e588594f2930c')

In [94]:
varList = [
    'CBBTCUSD', 'CBETHUSD' # Cryptocurrencies
]

BTC = {}
for var in varList:
    try:
        series_data = fred.get_series(var, observation_start=start_date, observation_end=end_date)
        BTC[var] = series_data
    except Exception as e:
        print(f"Error fetching data for {var}: {e}")

BTC_df = pd.DataFrame(BTC)
BTC_df.index.name = 'Date'

BTC_df.index = pd.to_datetime(BTC_df.index).strftime('%y%m%d')

BTC_df = BTC_df[BTC_df.index.isin(NVDA_stock_price_df.index)]

print(BTC_df.describe())


           CBBTCUSD     CBETHUSD
count    251.000000   251.000000
mean   51434.516972  2694.042351
std    14926.775117   719.164085
min    25161.680000  1539.260000
25%    38295.160000  2105.010000
50%    56684.730000  2640.160000
75%    64807.550000  3333.460000
max    73098.480000  4072.220000


In [95]:
varList = [
    'SP500', 'NASDAQCOM', 'DJIA' # Stock Market Indexes
]

fred_data = {}

for var in varList:
    try:
        # Fetch data from FRED API
        series_data = fred.get_series(var, observation_start=start_date, observation_end=end_date)
        
        # Convert the series to DataFrame
        fred_data[var] = series_data
    except Exception as e:
        print(f"Error fetching data for {var}: {e}")

# Combine all series into a single DataFrame
fred_df = pd.DataFrame(fred_data)
fred_df.index.name = 'Date'

# Format index to match the required date format: %y%m%d
fred_df.index = pd.to_datetime(fred_df.index).strftime('%y%m%d')

# Ensure dates align with a reference DataFrame (e.g., NVDA_stock_price_df)
reference_dates = NVDA_stock_price_df.index  # Replace with actual NVDA_stock_price_df when available
fred_df = fred_df[fred_df.index.isin(reference_dates)]

# Display descriptive statistics
print(fred_df.describe())


             SP500     NASDAQCOM          DJIA
count   251.000000    251.000000    251.000000
mean   4967.770398  15647.563386  37650.591394
std     428.518699   1553.565219   2314.278918
min    4117.370000  12595.610000  32417.590000
25%    4563.260000  14262.175000  36002.660000
50%    5051.410000  15865.250000  38503.690000
75%    5305.380000  16778.385000  39142.545000
max    5667.200000  18647.450000  41563.080000


In [96]:
feature = pd.concat([NVDA_stock_price_df, competitors_stock_price_df, ADS, FF5, BTC_df, fred_df], axis=1)
feature.to_csv('featurePrepared.csv')